In [1]:
#Tukish POS Tagging
import re
from zemberek.morphology import TurkishMorphology
from zemberek.tokenization import TurkishTokenizer

def pos_tagger(input, output):
    tokenizer = TurkishTokenizer.DEFAULT
    morphology = TurkishMorphology.create_with_defaults()
    
    input_file_path = input
    output_file_path = output
    
    with open(output_file_path, "w") as output_file:
        with open(input_file_path, "r") as input_file:
            line_count = 0
            error_count = 0
            
            for line in input_file:
                line_count += 1
                tags = []
                data = line.strip()
                tokens = tokenizer.tokenize(data)
                
                for token in tokens:
                    analysis = morphology.analyze(token.normalized)
                    if analysis.analysis_results:
                        match = re.search(r"\[(.*?):(.*?)(?=[\]\.,;!?]|$)", str(analysis.analysis_results[0]))
                        if match:
                            tag = match.group(2)
                            tag = re.sub(r'[^\w\s]', '', str(tag))
                            tags.append(str(tag) + " ")
                    else:
                        tags.append("X ")
                
                if len(tokens) != len(tags):
                    error_count += 1
                    print(f"Error in line {line_count}: Token count ({len(tokens)}) does not match tag count ({len(tags)})")
    
                output_file.write("".join(tags) + "\n")
    
            print(f"Total lines processed: {line_count}")
            print(f"Total errors found: {error_count}")


In [2]:
pos_tagger("TED2020.en-tr.tr-filtered.tr.train","TED2020_pos_tags_train")
pos_tagger("TED2020.en-tr.tr-filtered.tr.dev","TED2020_pos_tags_dev")
pos_tagger("TED2020.en-tr.tr-filtered.tr.test","TED2020_pos_tags_test")

2024-12-06 15:17:08,720 - zemberek.morphology.turkish_morphology - INFO
Msg: TurkishMorphology instance initialized in 12.091636180877686

Total lines processed: 337547
Total errors found: 0
2024-12-06 16:40:14,241 - zemberek.morphology.turkish_morphology - INFO
Msg: TurkishMorphology instance initialized in 11.944702386856079

Total lines processed: 10000
Total errors found: 0
2024-12-06 16:42:57,374 - zemberek.morphology.turkish_morphology - INFO
Msg: TurkishMorphology instance initialized in 13.04245138168335

Total lines processed: 10000
Total errors found: 0


In [ ]:
# TED2020 tr vocab
!onmt-build-vocab --sentencepiece model_type=bpe --size 32000 --save_vocab tr_vocab TED2020.en-tr.tr-filtered.tr.train

In [ ]:
#TED2020 en vocab
!onmt-build-vocab --sentencepiece model_type=bpe --size 32000 --save_vocab en_vocab TED2020.en-tr.en-filtered.en.train

In [1]:
# Turkish side subwording and subword units POS Tags
import re
import sentencepiece as spm

def preprocess_text(text, pos_tags):

    text = re.sub(r"([.,!?;:()\"'])", r" \1 ", text)
    text = re.sub(r"\s{2,}", " ", text).strip()
    
    words = text.split()
    
    new_pos_tags = []
    i = 0  # pos_tags index'i
    for word in words:
        #if re.match(r"[.,!?;:()\"']", word):
            #new_pos_tags.append('Punc') 
            #pass
        #else:
            if i < len(pos_tags):
                new_pos_tags.append(pos_tags[i])
                i += 1

    return words, new_pos_tags

def tokenize_and_label_separate_files(input_text_path, input_pos_path, output_token_path, output_pos_path, model_path):

    sp = spm.SentencePieceProcessor()
    sp.Load(model_path)
    
    with open(input_text_path, 'r', encoding='utf-8') as text_file, \
         open(input_pos_path, 'r', encoding='utf-8') as pos_file, \
         open(output_token_path, 'w', encoding='utf-8') as token_file, \
         open(output_pos_path, 'w', encoding='utf-8') as pos_file_out:
        
        for text_line, pos_line in zip(text_file, pos_file):
            pos_tags = pos_line.strip().split()
            preprocessed_words, adjusted_pos_tags = preprocess_text(text_line, pos_tags)
            
            tokenized_text = []
            tokenized_tags = []
            
            for word, tag in zip(preprocessed_words, adjusted_pos_tags):
                tokens = sp.EncodeAsPieces(word)
                tokenized_text.extend(tokens)
                tokenized_tags.extend([tag] * len(tokens))
            
            token_file.write(" ".join(tokenized_text) + "\n")
            pos_file_out.write(" ".join(tokenized_tags) + "\n")


input_text_path = "TED2020.en-tr.tr-filtered.tr.train"  
input_pos_path = "TED2020_pos_tags_train"  
output_token_path = "TED2020_tokens_train"  
output_pos_path = "TED2020_pos_tags_train.txt"  
model_path = "tr_vocab.model" 

tokenize_and_label_separate_files(input_text_path, input_pos_path, output_token_path, output_pos_path, model_path)


In [2]:
input_text_path = "TED2020.en-tr.tr-filtered.tr.dev"  
input_pos_path = "TED2020_pos_tags_dev"  
output_token_path = "TED2020_tokens_dev"  
output_pos_path = "TED2020_pos_tags_dev.txt"  
model_path = "tr_vocab.model"  

tokenize_and_label_separate_files(input_text_path, input_pos_path, output_token_path, output_pos_path, model_path)

input_text_path = "TED2020.en-tr.tr-filtered.tr.test"  
input_pos_path = "TED2020_pos_tags_test"  
output_token_path = "TED2020_tokens_test"  
output_pos_path = "TED2020_pos_tags_test.txt"  
model_path = "tr_vocab.model"  

tokenize_and_label_separate_files(input_text_path, input_pos_path, output_token_path, output_pos_path, model_path)

In [ ]:
# Turkish side shared vocab subwording and subword units POS Tags
input_text_path = "TED2020.en-tr.tr-filtered.tr.train"  
input_pos_path = "TED2020_pos_tags_train"  
output_token_path = "TED2020_tokens_train_shared"  
output_pos_path = "TED2020_pos_tags_train_shared.txt"  
model_path = "kk_tr_shared_vocab.model"  

tokenize_and_label_separate_files(input_text_path, input_pos_path, output_token_path, output_pos_path, model_path)

input_text_path = "TED2020.en-tr.tr-filtered.tr.dev" 
input_pos_path = "TED2020_pos_tags_dev"  
output_token_path = "TED2020_tokens_dev_shared"  
output_pos_path = "TED2020_pos_tags_dev-shared.txt"  
model_path = "kk_tr_shared_vocab.model" 

tokenize_and_label_separate_files(input_text_path, input_pos_path, output_token_path, output_pos_path, model_path)

input_text_path = "TED2020.en-tr.tr-filtered.tr.test"  
input_pos_path = "TED2020_pos_tags_test"  
output_token_path = "TED2020_tokens_test_shared"  
output_pos_path = "TED2020_pos_tags_test_shared.txt"  
model_path = "kk_tr_shared_vocab.model"  

tokenize_and_label_separate_files(input_text_path, input_pos_path, output_token_path, output_pos_path, model_path)


In [8]:
#Kazakh shared vocab subwording and subword units POS Tags
import re
import sentencepiece as spm

def preprocess_text(text, pos_tags):

    text = re.sub(r"([.,!?;:()\"'])", r" \1 ", text)
    text = re.sub(r"\s{2,}", " ", text).strip()
    
    words = text.split()
    
    new_pos_tags = []
    i = 0  # pos_tags index'i
    for word in words:
        #if re.match(r"[.,!?;:()\"']", word):
            #new_pos_tags.append('Punc')  
            #pass
        #else:
            if i < len(pos_tags):
                new_pos_tags.append(pos_tags[i])  
                i += 1

    return words, new_pos_tags

def tokenize_and_label_separate_files(input_text_path, input_pos_path, output_token_path, output_pos_path, model_path):

    sp = spm.SentencePieceProcessor()
    sp.Load(model_path)
    
    with open(input_text_path, 'r', encoding='utf-8') as text_file, \
         open(input_pos_path, 'r', encoding='utf-8') as pos_file, \
         open(output_token_path, 'w', encoding='utf-8') as token_file, \
         open(output_pos_path, 'w', encoding='utf-8') as pos_file_out:
        
        for text_line, pos_line in zip(text_file, pos_file):
            pos_tags = pos_line.strip().split()
            preprocessed_words, adjusted_pos_tags = preprocess_text(text_line, pos_tags)
            
            tokenized_text = []
            tokenized_tags = []
            
            for word, tag in zip(preprocessed_words, adjusted_pos_tags):
                tokens = sp.EncodeAsPieces(word)
                tokenized_text.extend(tokens)
                tokenized_tags.extend([tag] * len(tokens))
            
            token_file.write(" ".join(tokenized_text) + "\n")
            pos_file_out.write(" ".join(tokenized_tags) + "\n")


# Parametreler
input_text_path = "kk_train_shuffled.txt-filtered.kk" 
input_pos_path = "RoBERTa_KK_POS_train"  
output_token_path = "KK_tokens_train_shared" 
output_pos_path = "KK_pos_tags_train_shared.txt"  
model_path = "kk_tr_shared_vocab.model" 

tokenize_and_label_separate_files(input_text_path, input_pos_path, output_token_path, output_pos_path, model_path)


In [9]:
input_text_path = "kk_valid_shuffled.txt-filtered.kk" 
input_pos_path = "RoBERTa_KK_POS_valid" 
output_token_path = "KK_tokens_valid_shared"  
output_pos_path = "KK_pos_tags_valid_shared.txt"  
model_path = "kk_tr_shared_vocab.model"  

tokenize_and_label_separate_files(input_text_path, input_pos_path, output_token_path, output_pos_path, model_path)

input_text_path = "kk_test_shuffled.txt-filtered.kk"  
input_pos_path = "RoBERTa_KK_POS_test"  
output_token_path = "KK_tokens_test_shared"  
output_pos_path = "KK_pos_tags_test_shared.txt" 
model_path = "kk_tr_shared_vocab.model" 

tokenize_and_label_separate_files(input_text_path, input_pos_path, output_token_path, output_pos_path, model_path)

In [11]:
# TED2020 English side subwording
import re
import sentencepiece as spm

sp_model = spm.SentencePieceProcessor()
sp_model.Load("en_vocab.model")

def preprocess(text):

    text = re.sub(r'([.,!?;:()])', r' \1 ', text)

    text = re.sub(r'\s+', ' ', text)
    return text.strip()

def tokenize_and_save(input_file, model, output_file):
    with open(input_file, 'r', encoding='utf-8') as f, open(output_file, 'w', encoding='utf-8') as fo:
        for line in f:

            preprocessed_line = preprocess(line.strip())
            tokens = model.encode_as_pieces(preprocessed_line)
            fo.write(" ".join(tokens) + "\n")

tokenize_and_save("TED2020.en-tr.en-filtered.en.train", sp_model, "TED2020_train_target_tokens.txt")
tokenize_and_save("TED2020.en-tr.en-filtered.en.dev", sp_model, "TED2020_dev_target_tokens.txt")
tokenize_and_save("TED2020.en-tr.en-filtered.en.test", sp_model, "TED2020_test_target_tokens.txt")

In [12]:
#TED2020 english side shared vocab subwording
import re
import sentencepiece as spm

sp_model = spm.SentencePieceProcessor()
sp_model.Load("en_shared_vocab.model")

def preprocess(text):

    text = re.sub(r'([.,!?;:()])', r' \1 ', text)

    text = re.sub(r'\s+', ' ', text)
    return text.strip()

def tokenize_and_save(input_file, model, output_file):
    with open(input_file, 'r', encoding='utf-8') as f, open(output_file, 'w', encoding='utf-8') as fo:
        for line in f:

            preprocessed_line = preprocess(line.strip())
            tokens = model.encode_as_pieces(preprocessed_line)
            fo.write(" ".join(tokens) + "\n")

tokenize_and_save("TED2020.en-tr.en-filtered.en.train", sp_model, "TED2020_train_target_tokens_shared.txt")
tokenize_and_save("TED2020.en-tr.en-filtered.en.dev", sp_model, "TED2020_dev_target_tokens_shared.txt")
tokenize_and_save("TED2020.en-tr.en-filtered.en.test", sp_model, "TED2020_test_target_tokens_shared.txt")

In [13]:
# Kazakh corpus English side shared vocab subwording
import re
import sentencepiece as spm

sp_model = spm.SentencePieceProcessor()
sp_model.Load("en_shared_vocab.model")

def preprocess(text):

    text = re.sub(r'([.,!?;:()])', r' \1 ', text)

    text = re.sub(r'\s+', ' ', text)
    return text.strip()

def tokenize_and_save(input_file, model, output_file):
    with open(input_file, 'r', encoding='utf-8') as f, open(output_file, 'w', encoding='utf-8') as fo:
        for line in f:

            preprocessed_line = preprocess(line.strip())
            tokens = model.encode_as_pieces(preprocessed_line)
            fo.write(" ".join(tokens) + "\n")

tokenize_and_save("en_train_shuffled.txt-filtered.en", sp_model, "KK_train_target_tokens_shared.txt")
tokenize_and_save("en_valid_shuffled.txt-filtered.en", sp_model, "KK_valid_target_tokens_shared.txt")
tokenize_and_save("en_test_shuffled.txt-filtered.en", sp_model, "KK_test_target_tokens_shared.txt")

In [21]:
# Tr-En (TED2020)(POS Tags) -> Kk-En (POS Tags)
!onmt-main --model kk-tr-en-modelim.py --config kk-tr-en-pos.yml --auto_config train --with_eval --num_gpus 2

2024-12-09 11:33:09.892036: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-09 11:33:10.708480: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2024-12-09 11:33:10.708546: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2024-12-09 11:33:10.708554: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like t

In [22]:
# Tr-En (TED2020)(POS Tags) -> Kk-En (POS Tags)
!onmt-main --model kk-tr-en-modelim.py --config kk-tr-en-pos-2.yml --auto_config --checkpoint_path POS_TR_KK_EN/ckpt-100000 train --with_eval --num_gpus 2

2024-12-10 10:17:05.140603: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-10 10:17:06.272511: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2024-12-10 10:17:06.272698: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2024-12-10 10:17:06.272707: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like t

In [23]:
!CUDA_VISIBLE_DEVICES=0 onmt-main --config kk-tr-en-pos-2.yml --auto_config --checkpoint_path POS_TR_KK_EN-2/ckpt-100000 infer --features_file KK_tokens_test_shared KK_pos_tags_test_shared.txt --predictions_file output_tr_kk_en_pos.txt

2024-12-11 09:46:47.783269: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-11 09:46:48.933978: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2024-12-11 09:46:48.934076: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2024-12-11 09:46:48.934088: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like t

In [ ]:
!python3 MT-Preparation/subwording/3-desubword.py en_shared_vocab.model output_tr_kk_en_pos.txt

In [1]:
# BLEU and chrF scores
!python3 compute-bleu.py en_test_shuffled.txt-filtered.en output_tr_kk_en_pos.txt.desubword

Reference first sentence: In the developed world, this figure is 35 25%
Translated first sentence: In developed countries of the world , this figure is 35-25%
That's 100 lines that end in a tokenized period ('.')
It looks like you forgot to detokenize your test data, which may hurt your score.
If you insist your data is detokenized, or don't care, you can suppress this message with the `force` parameter.
BLEU2:  BLEU = 50.30 70.0/54.4/44.7/37.6 (BP = 1.000 ratio = 1.161 hyp_len = 480977 ref_len = 414303)
CHRF:  chrF2 = 76.38


In [26]:
# Average METEOR score (Ortalama METEOR Puanı)
import nltk
from nltk.translate.meteor_score import meteor_score

def read_and_tokenize_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()
    return [nltk.word_tokenize(line.strip()) for line in lines]

def calculate_meteor(reference_file, hypothesis_file):
    references = read_and_tokenize_file(reference_file)
    hypotheses = read_and_tokenize_file(hypothesis_file)
    
    if len(references) != len(hypotheses):
        raise ValueError("Dosyaların satır sayıları eşleşmiyor")

    total_meteor_score = 0.0
    for ref, hyp in zip(references, hypotheses):
        total_meteor_score += meteor_score([ref], hyp)

    average_meteor_score = total_meteor_score / len(references)
    return average_meteor_score

reference_file = 'en_test_shuffled.txt-filtered.en'
hypothesis_file = 'output_tr_kk_en_pos.txt.desubword'

score = calculate_meteor(reference_file, hypothesis_file)
print(f"Ortalama METEOR Puanı: {score}")


Ortalama METEOR Puanı: 0.7624199636634235


In [14]:
# Kk-En (POS Tags) -> Tr-En (TED2020)(POS Tags)
!onmt-main --model kk-tr-en-modelim.py --config kk-tr-en-pos-asıl.yml --auto_config train --with_eval --num_gpus 2

2024-12-07 20:14:22.507252: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-07 20:14:23.340587: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2024-12-07 20:14:23.340652: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2024-12-07 20:14:23.340660: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like t

In [15]:
# Kk-En (POS Tags) -> Tr-En (TED2020)(POS Tags)
!onmt-main --model kk-tr-en-modelim.py --config kk-tr-en-pos-asıl-2.yml --auto_config --checkpoint_path POS_KK_TR_EN/ckpt-100000 train --with_eval --num_gpus 2

2024-12-08 16:33:07.713258: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-08 16:33:08.902164: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2024-12-08 16:33:08.902228: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2024-12-08 16:33:08.902236: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like t

In [17]:
!CUDA_VISIBLE_DEVICES=0 onmt-main --config kk-tr-en-pos-asıl-2.yml --auto_config --checkpoint_path POS_KK_TR_EN_2/ckpt-100000 infer --features_file TED2020_tokens_test_shared TED2020_pos_tags_test_shared.txt --predictions_file output_kk_tr_en_pos.txt

2024-12-09 10:49:08.762012: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-09 10:49:09.555017: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2024-12-09 10:49:09.555111: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2024-12-09 10:49:09.555125: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like t

In [ ]:
!python3 MT-Preparation/subwording/3-desubword.py en_shared_vocab.model output_kk_tr_en_pos.txt

In [2]:
# BLEU and chrF scores
!python3 compute-bleu.py TED2020.en-tr.en-filtered.en.test output_kk_tr_en_pos.txt.desubword

Reference first sentence: It's quite widespread.
Translated first sentence: And that's a pretty common theme .
That's 100 lines that end in a tokenized period ('.')
It looks like you forgot to detokenize your test data, which may hurt your score.
If you insist your data is detokenized, or don't care, you can suppress this message with the `force` parameter.
BLEU2:  BLEU = 21.50 50.2/26.8/16.1/9.9 (BP = 1.000 ratio = 1.193 hyp_len = 236101 ref_len = 197923)
CHRF:  chrF2 = 50.69


In [20]:
# Average METEOR score (Ortalama METEOR Puanı)
import nltk
from nltk.translate.meteor_score import meteor_score

def read_and_tokenize_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()
    return [nltk.word_tokenize(line.strip()) for line in lines]

def calculate_meteor(reference_file, hypothesis_file):
    references = read_and_tokenize_file(reference_file)
    hypotheses = read_and_tokenize_file(hypothesis_file)
    
    if len(references) != len(hypotheses):
        raise ValueError("Dosyaların satır sayıları eşleşmiyor")

    total_meteor_score = 0.0
    for ref, hyp in zip(references, hypotheses):
        total_meteor_score += meteor_score([ref], hyp)

    average_meteor_score = total_meteor_score / len(references)
    return average_meteor_score

reference_file = 'TED2020.en-tr.en-filtered.en.test'
hypothesis_file = 'output_kk_tr_en_pos.txt.desubword'

score = calculate_meteor(reference_file, hypothesis_file)
print(f"Ortalama METEOR Puanı: {score}")

Ortalama METEOR Puanı: 0.5678681502396242


In [3]:
# Tr-En (TED2020)(base model)
!onmt-main --model kk-tr-en-shared.py --config tr-en.yml --auto_config train --with_eval --num_gpus 2

2024-12-12 10:35:15.838710: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-12 10:35:16.638176: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2024-12-12 10:35:16.638240: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2024-12-12 10:35:16.638248: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like t

In [4]:
!CUDA_VISIBLE_DEVICES=0 onmt-main --config tr-en.yml --auto_config --checkpoint_path TR-EN_std/ckpt-100000 infer --features_file TED2020_tokens_test --predictions_file output_tr_en_std.txt

2024-12-13 10:40:57.470147: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-13 10:40:58.616259: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2024-12-13 10:40:58.616465: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2024-12-13 10:40:58.616476: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like t

In [ ]:
!python3 MT-Preparation/subwording/3-desubword.py en_vocab.model output_tr_en_std.txt

In [3]:
# BLEU and chrF scores
!python3 compute-bleu.py TED2020.en-tr.en-filtered.en.test output_tr_en_std.txt.desubword

Reference first sentence: It's quite widespread.
Translated first sentence: This is very common .
That's 100 lines that end in a tokenized period ('.')
It looks like you forgot to detokenize your test data, which may hurt your score.
If you insist your data is detokenized, or don't care, you can suppress this message with the `force` parameter.
BLEU2:  BLEU = 22.87 55.0/28.8/16.9/10.2 (BP = 1.000 ratio = 1.058 hyp_len = 209406 ref_len = 197923)
CHRF:  chrF2 = 50.05


In [7]:
# Average METEOR score (Ortalama METEOR Puanı)
import nltk
from nltk.translate.meteor_score import meteor_score

def read_and_tokenize_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()
    return [nltk.word_tokenize(line.strip()) for line in lines]

def calculate_meteor(reference_file, hypothesis_file):
    references = read_and_tokenize_file(reference_file)
    hypotheses = read_and_tokenize_file(hypothesis_file)
    
    if len(references) != len(hypotheses):
        raise ValueError("Dosyaların satır sayıları eşleşmiyor")

    total_meteor_score = 0.0
    for ref, hyp in zip(references, hypotheses):
        total_meteor_score += meteor_score([ref], hyp)

    average_meteor_score = total_meteor_score / len(references)
    return average_meteor_score

reference_file = 'TED2020.en-tr.en-filtered.en.test'
hypothesis_file = 'output_tr_en_std.txt.desubword'

score = calculate_meteor(reference_file, hypothesis_file)
print(f"Ortalama METEOR Puanı: {score}")

Ortalama METEOR Puanı: 0.5538603230976438


In [8]:
# Tr-En (TED2020)(POS Tags)
!onmt-main --model kk-tr-en-modelim.py --config tr-en-pos.yml --auto_config train --num_gpus 2

2024-12-13 10:48:12.354368: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-13 10:48:13.140151: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2024-12-13 10:48:13.140217: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2024-12-13 10:48:13.140225: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like t

In [9]:
!CUDA_VISIBLE_DEVICES=0 onmt-main --config tr-en-pos.yml --auto_config --checkpoint_path POS_TR_EN/ckpt-100000 infer --features_file TED2020_tokens_test TED2020_pos_tags_test.txt --predictions_file output_tr_en_pos.txt

2024-12-15 16:09:32.942911: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-15 16:09:33.988874: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2024-12-15 16:09:33.988963: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2024-12-15 16:09:33.988973: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like t

In [ ]:
!python3 MT-Preparation/subwording/3-desubword.py en_vocab.model output_tr_en_pos.txt

In [4]:
# BLEU and chrF scores
!python3 compute-bleu.py TED2020.en-tr.en-filtered.en.test output_tr_en_pos.txt.desubword

Reference first sentence: It's quite widespread.
Translated first sentence: And this is a really common topic .
That's 100 lines that end in a tokenized period ('.')
It looks like you forgot to detokenize your test data, which may hurt your score.
If you insist your data is detokenized, or don't care, you can suppress this message with the `force` parameter.
BLEU2:  BLEU = 23.92 55.9/29.8/17.9/11.0 (BP = 1.000 ratio = 1.049 hyp_len = 207651 ref_len = 197923)
CHRF:  chrF2 = 50.51


In [13]:
# Average METEOR score (Ortalama METEOR Puanı)
import nltk
from nltk.translate.meteor_score import meteor_score

def read_and_tokenize_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()
    return [nltk.word_tokenize(line.strip()) for line in lines]

def calculate_meteor(reference_file, hypothesis_file):
    references = read_and_tokenize_file(reference_file)
    hypotheses = read_and_tokenize_file(hypothesis_file)
    
    if len(references) != len(hypotheses):
        raise ValueError("Dosyaların satır sayıları eşleşmiyor")

    total_meteor_score = 0.0
    for ref, hyp in zip(references, hypotheses):
        total_meteor_score += meteor_score([ref], hyp)

    average_meteor_score = total_meteor_score / len(references)
    return average_meteor_score

reference_file = 'TED2020.en-tr.en-filtered.en.test'
hypothesis_file = 'output_tr_en_pos.txt.desubword'

score = calculate_meteor(reference_file, hypothesis_file)
print(f"Ortalama METEOR Puanı: {score}")

Ortalama METEOR Puanı: 0.5618200915177892
